In [ ]:
import pandas as pd
import ipywidgets as widgets
import datetime as dt
from datetime import datetime
from IPython.display import display, clear_output
from IPython.core.display import HTML
import random
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import tempfile
import numpy as np
from functools import partial
import re

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
# AVISO PROMOTOR E PRACA ATIVA NAO FUNCIONA 

In [ ]:
class app:
    def __init__(self, main) -> None:
        self.main = main
        
    def _build_layout(self):
        self.output_main = widgets.Output()
        self.layout = widgets.VBox(children=[self.output_main])
        return self.layout
    
    def _filtrar_pedidos(self, status = None, statusIs = '=='):
        self._atualizar_filtros()
        pedidos = self.main.pedidos.query(
            f"@self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_praca in @self.pracas")
        if status != None:
            pedidos = pedidos.query(f"status {statusIs} @status")
        return pedidos
    
    def _filtrar_artigosNoPedido(self, status = "FECHADO"):
        self._atualizar_filtros()
        artigosNoPedido =     self.main.artigosPedido.query(
            f"ref in @self.artigosFiltrar and @self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_praca in @self.pracas and status == @status"
        )
        return artigosNoPedido
        
    def _filtrar_desempenho(self):
        self._atualizar_filtros()
        desempenho = self.main.desempenhoPraca.query("@self.dataInicial <= ano_mes_cobranca <= @self.dataFinal  and id_vendedor in @self.promotores and id_praca in @self.pracas")
        return desempenho
    
    def _get_listaVendedorasAtivas(self):
        pedidos = self._filtrar_pedidos('DIGITADO')
        lista_vendedorasAtivas = pedidos['id_sacoleira'].tolist()
        return lista_vendedorasAtivas
    
    def _get_itensVendedorasAtivas(self):
        listaAtiva = self._get_listaVendedorasAtivas()
        histArtigosAtivas = self.main.artigosPedido.query("id_vendedora in @listaAtiva")
        return histArtigosAtivas
    
    def _atualizar_exibicao(self, exibicoes, output=None):
        """
        Args:
            exibicoes (list): Uma lista contendo um ou mais dicionario de exibicao,
            exibicao (dic): Objeto a ser exibido em uma tab sendo seus argumentos:
            * 'nome' : Nome da tab;
            * 'objeto' : Objeto a ser exibido;
            * 'style' : parametros de estilo para dataframes;
            output (_type_, optional): _description_. Defaults to None.
        """
        if output is not None:
            output = output
        else:
            output = self.output_main

        if not isinstance(exibicoes, list):
            exibicoes = [exibicoes]
            
        with output:
            clear_output()
            tab_contents = [widgets.Output() for _ in range(len(exibicoes))]
            children = [widgets.VBox([tab]) for tab in tab_contents]
            tab = widgets.Tab()
            tab.children = children
            for i, extra in enumerate(exibicoes):
                tab.set_title(i, exibicoes[i]["nome"])
            display(tab)

            for i, exibicao in enumerate(exibicoes):
                with tab_contents[i]:
                    if isinstance(exibicao['objeto'], list):
                        for item in exibicao['objeto']:
                            display(item)
                    elif isinstance(exibicao['objeto'], folium.Map):
                        display(HTML(f'<div style="width: 1600px; height: 700px;">{exibicao["objeto"]._repr_html_()}</div>'))
                    elif isinstance(exibicao['objeto'], pd.DataFrame):
                        def criar_funcao_atualizar(exibicao, dropdown_coluna, dropdown_ascDesc, output):
                            def atualizar_dataframe(change):
                                
                                dataframe = exibicao['objeto']
                                dataframe_final = dataframe.sort_values(by=dropdown_coluna.value, ascending=dropdown_ascDesc.value)
                                if "style" in exibicao:
                                    dataframe_final = dataframe_final.style.format(exibicao['style'])
                                with output:
                                    clear_output()
                                    display(dataframe_final)
                            return atualizar_dataframe
                        exibicao = exibicao
                        dropdown_coluna_exibicao = widgets.Dropdown(description='Campo', options=exibicao['objeto'].columns)
                        dropdown_ascDesc_exibicao = widgets.Dropdown(description='Orientação', options=[('Descendente', False), ('Ascendente', True)])
                        button_ordenar_exibicao = widgets.Button(description='Ordenar')
                        box_ordem_exibicao = widgets.HBox(children=[dropdown_coluna_exibicao, dropdown_ascDesc_exibicao, button_ordenar_exibicao])
                        button_exportar_exibicao = widgets.Button(description="Exportar")
                        output_exibicao = widgets.Output()
                        box_main = widgets.VBox(children=[box_ordem_exibicao, button_exportar_exibicao, output_exibicao])
                        funcao_atualizar_dataframe = criar_funcao_atualizar(exibicao, dropdown_coluna_exibicao, dropdown_ascDesc_exibicao, output_exibicao)
                        button_ordenar_exibicao.on_click(funcao_atualizar_dataframe)
                        display(box_main)
                        funcao_atualizar_dataframe(None)  # O segundo argumento (change) será preenchido automaticamente pelo widget
                    else:
                        display(exibicao['objeto'])

    def _atualizar_dataframe(self, dataframe, extras = [], objeto = None, style = None):
        if objeto != None:
            output = objeto
        else:
            output = self.output_main
        
        with output:
            clear_output()
            dropdown_coluna = widgets.Dropdown(description = 'Campo', options = dataframe.columns)
            dropdown_ascDesc = widgets.Dropdown(description = 'Orientação', options = [('Ascendente', True), ('Descendente', False)])
            button_ordenar = widgets.Button(description = 'Ordenar')
            box_ordem = widgets.HBox(children=[dropdown_coluna, dropdown_ascDesc, button_ordenar])
            button_exportar = widgets.Button(description="Exportar")
            output_df = widgets.Output()
            box_main = widgets.VBox(children=[box_ordem, button_exportar, output_df])
            #TABS
            tab_contents = [widgets.Output() for _ in range(len(extras) + 1)]
            children = [widgets.VBox([tab]) for tab in tab_contents]
            tab = widgets.Tab()
            tab.children = children
            tab.set_title(0, 'DataFrame')
            for i, extra in enumerate(extras):
                tab.set_title(i + 1, extras[i]["nome"])
            display(tab)
            
            def atualizar_dataframe():
                with output_df:
                    clear_output()
                # Aplicar estilo ao DataFrame
                dataframe_final = dataframe.sort_values(by=dropdown_coluna.value, ascending=dropdown_ascDesc.value)

                if style is not None:
                    dataframe_final = dataframe_final.style.format(style)
                else:
                    dataframe_final = dataframe_final

                with output_df:
                    display(dataframe_final)

            
            with tab_contents[0]:
                display(box_main)
                
            for i, extra in enumerate(extras):
                with tab_contents[i + 1]:
                    if isinstance(extra['objeto'], list):
                        for item in extra['objeto']:
                            display(item)
                    elif isinstance(extra['objeto'], folium.Map):
                        display(HTML(f'<div style="width: 1600px; height: 700px;">{extra["objeto"]._repr_html_()}</div>'))
                    elif isinstance(extra['objeto'], pd.DataFrame):
                        dropdown_coluna_extra = widgets.Dropdown(description = 'Campo', options = extra['objeto'])
                        dropdown_ascDesc_extra = widgets.Dropdown(description = 'Orientação', options = [('Ascendente', True), ('Descendente', False)])
                        button_ordenar_extra = widgets.Button(description = 'Ordenar')
                        box_ordem_extra = widgets.HBox(children=[dropdown_coluna_extra, dropdown_ascDesc_extra, button_ordenar_extra])
                        button_exportar_extra = widgets.Button(description="Exportar")
                        output_extra = widgets.Output()
                        box_main = widgets.VBox(children=[box_ordem_extra, button_exportar_extra, output_extra])
                        def atualizar_ordem_extra(obj):
                            extra['objeto'].sort_values(by= dropdown_coluna.value, ascending = dropdown_ascDesc.value, inplace = True) # type: ignore
                            self._atualizar_dataframe(dataframe, extras = extras)
                        button_ordenar.on_click(lambda b: atualizar_dataframe())
                    else:
                        display(extra['objeto'])

            def atualizar_ordem(s):
                dataframe.sort_values(by= dropdown_coluna.value, ascending = dropdown_ascDesc.value, inplace = True) # type: ignore
                self._atualizar_dataframe(dataframe, extras = extras)
            
            def on_button_click(b):
                agora = datetime.now()
                export = agora.strftime('%Y-%m-%d_%H-%M-%S')
                export_df = dataframe.reset_index() # type: ignore
                export_df.to_excel(f"export/dataframe_exportado_{export}.xlsx", index=True)
            
            atualizar_dataframe()
            button_ordenar.on_click(lambda b: atualizar_dataframe())
            button_exportar.on_click(on_button_click)    
    
    def mapaDeBarras(self, df, col):
        df = df.dropna(subset=['latitude', 'longitude'])
        marcadores = []
        altura_maxima = 100
        for index, row in df.iterrows():
            altura_barra = (row[col] / df[col].max()) * altura_maxima
            marcador = folium.Marker(
                location=[row['latitude'], row['longitude']],
                icon=folium.DivIcon(
                    icon_size=(10, altura_barra),
                    icon_anchor=(5, altura_barra / 2),
                    html=f'<div style="width: 10px; height: {altura_barra}px; background-color: blue;"></div>',
                ),
            )
            marcadores.append(marcador)
        return marcadores    
    
    def mapaDeCalor(self, df, col ):
        df = df.dropna(subset=['latitude', 'longitude'])
        min = df[col].min()
        max = df[col].max()
        df['scaled'] = (df[col] - min) / (max - min)
        heat_data = [[row['latitude'], row['longitude'], row['scaled']] for index, row in df.iterrows()]
        
        return HeatMap(heat_data)
    
    def criarMapa(self, df, mapsConfigs):
        """_summary_
        Args:
            mapsConfigs (dic): {type = tipo do mapa, col = coluna, kwargs = kwargs}
        """
        df = df.dropna(subset=['latitude', 'longitude'])
        m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=6)
        if isinstance(mapsConfigs, list):
            grupo_marcadores = folium.FeatureGroup(name='Marcadores')
            for mapConfig in mapsConfigs:
                tipo = getattr(self, mapConfig['type'])
                marcadores = tipo(df, mapConfig['col'])
                if isinstance(marcadores, list):
                    for marcador in marcadores:
                        grupo_marcadores.add_child(marcador)
                else:
                    grupo_marcadores.add_child(marcadores)
            grupo_marcadores.add_to(m)
            folium.LayerControl().add_to(m)
        else:
            funcao_selecionada = getattr(self, mapsConfigs['type'])
            marcador = funcao_selecionada(df, mapsConfigs['col'])
            if isinstance(marcador, list):
                for marc in marcador:
                    marc.add_to(m)
            else:
                marcador.add_to(m)

        return m
    
    def gerar_cor(self):
        return "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
    def _atualizar_filtros(self):
        self.artigosFiltrar = self.main.filtros['artigos']
        self.dataInicial = self.main.filtros['data inicial']
        self.dataFinal = self.main.filtros['data final']
        self.promotores = self.main.filtros['promotores']
        self.pracas = self.main.filtros['pracas']

In [ ]:
class cidadesApp(app):
    def __init__(self, main):
        super().__init__(main)
        self.listaFuncoes = [('Mapa de Praças','posicaoDasPracas'),
                            ('Distribuido', 'distribuicaoAtiva'), 
                            ('Vendido', 'faturamentoCidade'),
                            ('Distribuido X Vendido', 'faturamentoDistribuicao'),
                            ('Vendas (Quantidade)', 'vendasDeArtigos')]
    
    def posicaoDasPracas(self):
        def criarDataframe(dataframe):
            df = pedidos.groupby(['cidade', 'descricao_praca']).agg({'id_pedido': 'count', 'latitude': 'mean', 'longitude': 'mean'})
            return {'nome':'DataFrame', 'objeto':df}
        def criarMapa(dataframe):
            dfm = dataframe.reset_index()
            mapa = folium.Map(location=[df['objeto']['latitude'].mean(), df['objeto']['longitude'].mean()], zoom_start=10)
            for _, row in dfm.iterrows():
                praca_info = f"Descrição da Praça: {row['descricao_praca']}<br>Quantidade de Pedidos: {row['id_pedido']}"
                folium.Marker(
                    location=[row['latitude'], row['longitude']],
                    popup=praca_info,
                    icon=folium.Icon(icon='cloud')
                ).add_to(mapa)
            return {'nome':'Mapa de Calor', 'objeto':mapa}
                
        pedidos = self._filtrar_pedidos(status= "DIGITADO")
        df = criarDataframe(pedidos)
        mapa = criarMapa(df['objeto'])
        self._atualizar_exibicao([df,mapa])
        
    def vendasDeArtigos(self):
        artigosNoPedido = self._filtrar_artigosNoPedido(status= "FECHADO")
        mapa_calor = self.criarMapa(artigosNoPedido, {'type': 'mapaDeCalor', 'col':'quantidade_vendida'})        
        artigosNoPedido = artigosNoPedido.groupby('cidade').agg({'id_pedido': 'count','quantidade_vendida': 'sum'})
        self._atualizar_dataframe(artigosNoPedido,extras = [{'nome':'Mapa de Calor', 'objeto':mapa_calor}])
        
    def distribuicaoAtiva(self):
        pedidos = self._filtrar_pedidos(status= "DIGITADO")
        mapa_barras = self.criarMapa(pedidos, {'type': 'mapaDeBarras', 'col':'valor_total_pedido'})
        mapa_calor = self.criarMapa(pedidos, {'type': 'mapaDeCalor', 'col':'valor_total_pedido'})        
        pedidos = pedidos.groupby('cidade').agg({'id_sacoleira': 'count','valor_total_pedido': 'sum'})
        pedidos = pedidos[['id_sacoleira', 'valor_total_pedido']]
        pedidos.rename(columns={'id_sacoleira': 'Pedidos', 'valor_total_pedido':'Valor Distribuido'}, inplace=True)
        pedidos['Sacola Media PV'] = pedidos['Valor Distribuido']/pedidos['Pedidos']
        pedidos = pedidos[['Pedidos', 'Valor Distribuido', 'Sacola Media PV']]
        self._atualizar_dataframe(pedidos,extras = [{'nome':'Mapa de Calor', 'objeto':mapa_calor}])
        
    def faturamentoCidade(self):
        pedidos = self._filtrar_pedidos(status= "FECHADO")
        mapa_barras = self.criarMapa(pedidos, {'type': 'mapaDeBarras', 'col':'valor_venda_pedido'})
        mapa_calor = self.criarMapa(pedidos, {'type': 'mapaDeCalor', 'col':'valor_venda_pedido'})
        pedidos = pedidos.groupby('cidade').agg({'id_sacoleira': 'count','valor_venda_pedido': 'sum'})
        pedidos = pedidos[['id_sacoleira', 'valor_venda_pedido']]
        pedidos.rename(columns={'id_sacoleira': 'Pedidos', 'valor_venda_pedido':'Valor Vendido'}, inplace=True)
        pedidos['Sacola Media PV'] = pedidos['Valor Vendido']/pedidos['Pedidos']
        pedidos = pedidos[['Pedidos', 'Valor Vendido', 'Sacola Media PV']]
        self._atualizar_dataframe(pedidos,extras = [{'nome':'Mapa de Calor', 'objeto':mapa_calor}, {'nome':'Mapa de Barras', 'objeto':mapa_barras}])
        
    def faturamentoDistribuicao(self):
        pedidos = self._filtrar_pedidos(status= "FECHADO")
        mapaTeste = self.criarMapa(pedidos, [{'type': 'mapaDeBarras', 'col': 'valor_total_pedido'}, {'type': 'mapaDeCalor', 'col': 'valor_venda_pedido'}])
        mapa_distribuicao = self.criarMapa(pedidos, {'type': 'mapaDeCalor', 'col':'valor_total_pedido'})
        mapa_venda = self.criarMapa(pedidos, {'type': 'mapaDeCalor', 'col':'valor_venda_pedido'})
        pedidos = pedidos.groupby('cidade').agg({'id_sacoleira': 'count','valor_venda_pedido': 'sum'})
        pedidos = pedidos[['id_sacoleira', 'valor_venda_pedido']]
        pedidos.rename(columns={'id_sacoleira': 'Pedidos', 'valor_venda_pedido':'Valor Vendido'}, inplace=True)
        pedidos['Sacola Media PV'] = pedidos['Valor Vendido']/pedidos['Pedidos']
        pedidos = pedidos[['Pedidos', 'Valor Vendido', 'Sacola Media PV']]
        self._atualizar_dataframe(pedidos,extras = [{'nome': 'Teste', 'objeto': mapaTeste},{'nome':'Mapa de Distribuicao', 'objeto':mapa_distribuicao}, {'nome':'Mapa de Venda', 'objeto':mapa_venda}])

In [ ]:
class vendedorasApp(app):
    def __init__(self, main) -> None:
        super().__init__(main)
        self.listaFuncoes = [('Todas Vendedoras', 'analise_cobradas'),
                            ('Vendedoras Ativas', 'analise_ativas')]
        
    def _build_layout(self):
        self.selecao_vendedora = widgets.Dropdown(options=[],
                                description='Vendedora:',
                                disabled=False)
        self.selecao_vendedora.observe(self.vendedora_unica_selecionada, names='value') # type: ignore
        self.selecao_data_cobranca = widgets.Dropdown(options=[],
                                description='Data de Cobrança:')
        self.selecao_data_cobranca.observe(self._data_selecionada, names='value') # type: ignore
        #TODO FILTRO
        self.selecao_status = widgets.SelectMultiple(
            options=['FECHADO', 'DIGITADO', 'PEPINO'],
            value=['FECHADO', 'DIGITADO', 'PEPINO'],
            description='Status',
            disabled=False
        )
        self.box_vendedoras_todas_cabecalhos = widgets.HBox([self.selecao_data_cobranca, self.selecao_status])
        self.output_dataframe_vendedoras_todas_historico = widgets.Output()
        self.box_tab_vendedoras_todas = widgets.VBox([self.box_vendedoras_todas_cabecalhos, self.output_dataframe_vendedoras_todas_historico])
        self.dataframe_historico_vendedora = 0
        tab_vendedoras_contents = ['Todas']
        children = [self.box_tab_vendedoras_todas]
        self.tabs_vendedoras = widgets.Tab(children=children)
        for i, title in enumerate(tab_vendedoras_contents):
            self.tabs_vendedoras.set_title(i, title)
        self.layout = widgets.VBox(children=[self.selecao_vendedora, self.tabs_vendedoras])
        return self.layout
        
    def analise_ativas(self):
        pedidosDigitados = self._filtrar_pedidos('DIGITADO')
        listaAtivas = pedidosDigitados['id_sacoleira'].unique()
        pedidosTodos = self._filtrar_pedidos()
        pedidosTodos = pedidosTodos.query('id_sacoleira in @listaAtivas')
        self.pedidosTodos = pedidosTodos.merge(self.main.sacoleiras[['id_sacoleira', 'nome']], on='id_sacoleira')
        datasCobranca = pedidosTodos['ano_mes_cobranca'].unique()
        datasCobranca = sorted(datasCobranca, reverse=True)
        self.selecao_data_cobranca.options = datasCobranca
    
    def analise_cobradas(self):
        pedidosTodos = self._filtrar_pedidos()
        self.pedidosTodos = pedidosTodos.merge(self.main.sacoleiras[['id_sacoleira', 'nome']], on='id_sacoleira')
        datasCobranca = pedidosTodos['ano_mes_cobranca'].unique()
        datasCobranca = sorted(datasCobranca, reverse=True)
        self.selecao_data_cobranca.options = datasCobranca

    def _data_selecionada(self, value):
        self.dataDf = value['new']
        self.dataDf = self.dataDf.strftime("%m/%Y") # type: ignore
        # Convertendo as strings para o tipo datetime
        self.dataDf = datetime.strptime(self.dataDf, "%m/%Y").date()
        self.criar_dataframe_central()

    def vendedora_unica_selecionada(self, value):
        value = value['new']
        self.criar_tab_vendedora(value)

    def fechar_tab_vendedora(self, b):
        indice_tab_ativa = self.tabs_vendedoras.selected_index
        if indice_tab_ativa is not None and indice_tab_ativa >= 0:
            self.tabs_vendedoras.children = [
            child for i, child in enumerate(self.tabs_vendedoras.children) if i != indice_tab_ativa # type: ignore
        ]

    def criar_tab_vendedora(self, sacoleira):
        button_fechar_aba = widgets.Button(description= 'fechar aba')
        button_fechar_aba.on_click(self.fechar_tab_vendedora)
        output_vendedora = widgets.Output()
        nova_aba = widgets.VBox(children=[button_fechar_aba, output_vendedora])
        self.tabs_vendedoras.children = list(self.tabs_vendedoras.children) + [nova_aba] # type: ignore
        self.tabs_vendedoras.set_title(len(self.tabs_vendedoras.children) - 1, f'{self.selecao_vendedora.label}')
        self.criar_relatorios_vendedora(sacoleira, output_vendedora)

    def criar_relatorios_vendedora(self, sacoleira, output):
        def criarHistorico(sacoleira):
            dataframe_historico_vendedora = self.main.pedidos.query('id_sacoleira == @sacoleira')
            dataframe_historico_vendedora = dataframe_historico_vendedora[['ano_mes_cobranca', 'id_pedido', 'valor_total_pedido', 'valor_venda_pedido']].sort_values('id_pedido', ascending= True)
            dataframe_historico_vendedora['% Vendida'] = dataframe_historico_vendedora['valor_venda_pedido']/dataframe_historico_vendedora['valor_total_pedido']
            dataframe_historico_vendedora['id_pedido_anterior'] = dataframe_historico_vendedora['id_pedido'].shift(1)
            dataframe_historico_vendedora['Similaridade'] = 0.0
            for index, row in dataframe_historico_vendedora.iterrows():
                similaridade = self._do_calcular_similaridade_pedidos(row)
                dataframe_historico_vendedora.at[index, 'Similaridade'] = similaridade
            dataframe_historico_vendedora = dataframe_historico_vendedora[['ano_mes_cobranca', 'id_pedido', 'valor_total_pedido', 'valor_venda_pedido','% Vendida', 'Similaridade' ]]
            dataframe_historico_vendedora = dataframe_historico_vendedora.rename(columns={
                'ano_mes_cobranca': 'Ano Mês Cobrança',
                'id_pedido': 'Pedido',
                'valor_total_pedido': 'Total Deixado',
                'valor_venda_pedido': 'Valor vendido',
            })
            style = {
                'Pedido': '{:.0f}',
                'Total Deixado': 'R$ {:,.2f}',
                'Valor vendido': 'R$ {:,.2f}',
                'Similaridade': '{:.2%}',
                '% Vendida': '{:.2%}'
            }
            return {'nome':'Historico de Vendas', 'objeto': dataframe_historico_vendedora, 'style': style}
        def criarHistoricoVendas(sacoleira):
            itensFechados = self.main.artigosPedido.query(f'id_sacoleira == @sacoleira and status != "DIGITADO"')
            itensGroup = itensFechados.groupby(['ref', 'artigo'])
            df = pd.DataFrame()
            
            df['Quantidade Entregue'] = itensGroup['quantidade_entregue'].sum()
            df['Quantidade Vendida'] = itensGroup['quantidade_vendida'].sum()
            df['% Vendida'] = df['Quantidade Vendida']/df['Quantidade Entregue']
            df['Venda Mediana'] = itensGroup['quantidade_vendida'].mean()
            if self.main.artigosPedido.query(f'id_sacoleira == @sacoleira and status == "DIGITADO"').shape[0] > 0:
                itensDigitados = self.main.artigosPedido.query(f'id_sacoleira == @sacoleira and status == "DIGITADO"').groupby(['ref', 'artigo'])
                dfDigitado = pd.DataFrame()
                dfDigitado['Total Atual'] = itensDigitados['quantidade_entregue'].sum()
                df = df.merge(dfDigitado,  left_index=True, right_index=True, how='left')
            
            df.fillna(0, inplace=True)
            
            return {'nome':'Historico de Artigos', 'objeto': df}
        
        sacoleira = int(sacoleira)
        relatorioHistorico = criarHistorico(sacoleira)
        todosArtigos = criarHistoricoVendas(sacoleira)
        
        self._atualizar_exibicao([relatorioHistorico,todosArtigos], output=output)

    def _do_calcular_similaridade_pedidos(self, row):
        id_pedido1 = row['id_pedido']
        id_pedido2 = row['id_pedido_anterior']  # coluna que contém o id_pedido da linha anterior
        filtro = self.main.artigosPedido[self.main.artigosPedido['id_pedido'].isin([id_pedido1, id_pedido2])]
        referencias_pedido1 = filtro[filtro['id_pedido'] == id_pedido1]['ref'].tolist()
        referencias_pedido2 = filtro[filtro['id_pedido'] == id_pedido2]['ref'].tolist()
        # Calculando a similaridade de Jaccard entre as referências
        set_referencias1 = set(referencias_pedido1)
        set_referencias2 = set(referencias_pedido2)
        intersecao = set_referencias1.intersection(set_referencias2)
        uniao = set_referencias1.union(set_referencias2)
        similaridade = len(intersecao) / len(uniao)
        return similaridade       
        
    def criar_dataframe_central(self):
        def mainDf():
            dataframe_vendedoras_todas_historico = self.pedidosTodos
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico.query('ano_mes_cobranca == @self.dataDf')
            dataframe_vendedoras_todas_historico_copy = dataframe_vendedoras_todas_historico.copy()
            dataframe_vendedoras_todas_historico_copy['% Vendida'] = dataframe_vendedoras_todas_historico_copy['valor_venda_pedido'] / dataframe_vendedoras_todas_historico_copy['valor_total_pedido']
            dataframe_vendedoras_todas_historico_copy['Relevancia Cobrança'] = dataframe_vendedoras_todas_historico_copy['valor_venda_pedido'] / dataframe_vendedoras_todas_historico_copy['valor_venda_pedido'].sum()
            dataframe_vendedoras_todas_historico_copy['Relevancia Consignado'] = dataframe_vendedoras_todas_historico_copy['valor_total_pedido'] / dataframe_vendedoras_todas_historico_copy['valor_total_pedido'].sum()
            dataframe_vendedoras_todas_historico_copy['Ativa'] = dataframe_vendedoras_todas_historico_copy.apply(lambda row: 'sim' if any((self.pedidosTodos['id_sacoleira'] == row['id_sacoleira']) & (self.pedidosTodos['status'] == 'DIGITADO')) else 'não', axis=1)
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico_copy
            df_dadosExtra = self.main.pedidos.groupby('id_sacoleira').agg({'valor_venda_pedido': 'median', 'id_pedido': 'count'}).reset_index()
            df_dadosExtra = df_dadosExtra.rename(columns= {'valor_venda_pedido': 'Venda Mediana', 'id_pedido': 'Total Pedidos'})
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico.merge(df_dadosExtra, on = 'id_sacoleira')
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico[['id_sacoleira','nome', 'Ativa', 'Total Pedidos', 'status', 'valor_total_pedido', 'Relevancia Consignado', 'valor_venda_pedido','Relevancia Cobrança','% Vendida', 'Venda Mediana', 'descricao_praca']]
            
            dataframe_vendedoras_todas_historico = dataframe_vendedoras_todas_historico.rename(columns={
                'id_sacoleira': 'Id',
                'nome': 'Nome',
                'status': 'Status',
                'valor_total_pedido': 'Valor Pedido',
                'valor_venda_pedido': 'Valor Vendido',
                'descricao_praca': 'Praca'
            })
            style = {
                'Valor Pedido':'R$ {:,.2f}' ,
                'Valor Vendido':'R$ {:,.2f}',
                'Venda Mediana':'R$ {:,.2f}',
                '% Vendida':'{:.2%}',
                'Relevancia Cobrança':'{:.2%}',
                'Relevancia Consignado':'{:.2%}',
                'Total Pedidos': '{:.0f}'
            }
            return {'nome': 'Tabela', 'objeto': dataframe_vendedoras_todas_historico, 'style': style}
        def dfResumo(mainDf):
            datadf = pd.DataFrame()
            mainGb = mainDf.groupby('Praca')
            datadf['Vendedoras'] = mainGb['Total Pedidos'].count()
            datadf['Expectativa de Venda'] = mainGb['Venda Mediana'].sum()
            datadf['Total Vendido'] = mainGb['Valor Vendido'].sum()
            datadf['Resultado X Previsao'] = datadf['Total Vendido']/datadf['Expectativa de Venda']
            datadf['Venda Media'] = datadf['Total Vendido']/datadf['Vendedoras']
            datadf['Sacola Media'] = mainGb['Valor Pedido'].median()
            datadf['% Vendida Media'] = datadf['Venda Media']/datadf['Sacola Media']
            return {'nome': 'Resultado', 'objeto': datadf}
        
        dataframePrincipal = mainDf()
        dataframeResumo = dfResumo(dataframePrincipal['objeto'])
        self._atualizar_exibicao([dataframePrincipal, dataframeResumo], output= self.output_dataframe_vendedoras_todas_historico)
        self.carregar_lista_vendedoras(dataframePrincipal['objeto'])
            
    def carregar_lista_vendedoras(self, df):
        listaVendedoras = []
        df.sort_values(by= 'Id', ascending = True, inplace = True)
        for _, linha in df.iterrows():
            busca = linha['Id']
            sacoleira = df.query('Id == @busca')
            tupla = (f"{sacoleira['Id'].to_string(index=False)} - {sacoleira['Nome'].to_string(index=False)}", sacoleira['Id'].to_string(index=False))
            listaVendedoras.append(tupla)
        self.selecao_vendedora.options = listaVendedoras        

In [ ]:
class desempenhoApp(app):
    def __init__(self, main) -> None:
        super().__init__(main)
        self.listaFuncoes = [('Desempenho Periodo', 'desempenho_geral_mes'),
                            ('Desempenho Periodo (Promotor)', 'desempenho_promotor_mes'),
                            ('Desempenho Periodo (Praça)', 'desemepnho_praca_mes'),
                            ('Faturamento Periodo', 'faturamento_geral_mes'), 
                            ('Faturamento Promotores', 'faturamento_promotor_mes'), 
                            ('Faturamento Praças','faturamento_praca_mes'),  
                            ('Faturamento Promotor/Praças','faturamento_promotor_praca_mes')]
        
    def desempenho_geral_mes(self):
        def gerarDataframe(dados):
            desempenho = dados[['ano_mes_cobranca','pedidos_fechados','faturamento_liquido', 'total_novos_pepinos', 'vl_total_pedidos_liquido']].groupby('ano_mes_cobranca').sum()
            desempenho.reset_index(inplace = True)
            style = {
                'pedidos_fechados': '{:.0f}',
                'faturamento_liquido': 'R$ {:,.2f}',
                'total_novos_pepinos': 'R$ {:,.2f}',
                'vl_total_pedidos_liquido': '{:.2%}'}            
            return {'nome': 'Desempenho', 'objeto': desempenho, 'style': style}
        def gerarGraficoTempo(dados):
            desempenho = dados[['ano_mes_cobranca','pedidos_fechados','faturamento_liquido', 'total_novos_pepinos', 'vl_total_pedidos_liquido']].groupby('ano_mes_cobranca').sum()
            fig, ax1 = plt.subplots(figsize=(10, 6))
            ax1.plot(desempenho.index, desempenho['faturamento_liquido'], marker='^', linestyle='-', color='green', label='Faturamento Líquido')
            ax1.set_xlabel('Ano e Mês de Cobrança')
            ax1.set_ylabel('Faturamento Líquido')
            ax1.tick_params(axis='y')
            ax1.plot(desempenho.index, desempenho['total_novos_pepinos'], marker='v', linestyle='-', color='red', label='Total Novos Pepinos')
            ax2 = ax1.twinx()
            ax2.plot(desempenho.index, desempenho['vl_total_pedidos_liquido'], marker='D', linestyle='--', color='blue', label='VL Total Pedidos Líquido')
            ax2.set_ylabel('VL Total Pedidos Líquido')
            ax2.tick_params(axis='y')
            lines, labels = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax2.legend(lines + lines2, labels + labels2, loc="upper left")
            ax1.set_title('Desempenho ao Longo do Tempo')
            ax1.grid(True)
            plt.xticks(rotation=45)
            meu_grafico = plt.gcf()
            plt.close()
            return {'nome': 'Grafico Tempo', 'objeto': meu_grafico}
        
        dados = self._filtrar_desempenho()
        desempenho = gerarDataframe(dados)
        graficoTempo = gerarGraficoTempo(dados)
        self._atualizar_exibicao([desempenho,graficoTempo])

    def desempenho_promotor_mes(self):
        pass
    
    def desemepnho_praca_mes(self):
        pass
        
    def faturamento_geral_mes(self):
        desempenho = self._filtrar_desempenho()
        totalFaturado = desempenho[['ano_mes_cobranca','faturamento_liquido']].groupby('ano_mes_cobranca').sum()
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(totalFaturado.index, totalFaturado['faturamento_liquido'], marker='o', linestyle='-')
        ax.set_title('Faturamento Líquido ao Longo do Tempo')
        ax.set_xlabel('Ano e Mês de Cobrança')
        ax.set_ylabel('Faturamento Líquido')
        ax.grid(True)
        plt.xticks(rotation=45)
        x = np.arange(len(totalFaturado.index))
        y = totalFaturado['faturamento_liquido']
        coefficients = np.polyfit(x, y, 1)
        trendline = np.poly1d(coefficients)
        ax.plot(totalFaturado.index, trendline(x), linestyle='--', color='red', label='Linha de Tendência')
        plt.tight_layout()
        meu_grafico = plt.gcf()
        plt.close()
        self._atualizar_exibicao([{'nome':'Total Faturado', 'objeto':totalFaturado},{'nome':'Gráfico', 'objeto':meu_grafico}])
        
    def faturamento_promotor_mes(self):
        desempenho = self._filtrar_desempenho()
        totalFaturado = desempenho[['ano_mes_cobranca','faturamento_liquido', 'descricao_vendedor']].groupby(['ano_mes_cobranca', 'descricao_vendedor']).sum()
        totalFaturado = totalFaturado.sort_values(by= ['ano_mes_cobranca', 'faturamento_liquido'], ascending= False)
        pivot_df = totalFaturado.pivot_table(index='ano_mes_cobranca', columns='descricao_vendedor', values='faturamento_liquido', fill_value=0)
        col_order = pivot_df.sum().sort_values(ascending=False).index
        pivot_df = pivot_df[col_order]
        fig, ax = plt.subplots(figsize=(12, 6))
        num_vendedores = len(pivot_df.columns)
        colors = ['#%02x%02x%02x' % (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(num_vendedores)]
        bottom = None
        for vendedor, color in zip(pivot_df.columns, colors):
            ax.bar(
                pivot_df.index,
                pivot_df[vendedor],
                label=vendedor,
                bottom=bottom, # type: ignore
                color=color,
                alpha=1,
                width=10.0,
            )
            if bottom is None:
                bottom = pivot_df[vendedor]
            else:
                bottom += pivot_df[vendedor]
        ax.set_title('Faturamento Líquido por Vendedor ao Longo do Tempo')
        ax.set_xlabel('Ano e Mês de Cobrança')
        ax.set_ylabel('Faturamento Líquido')
        ax.grid(True)  # Adicionar grades
        ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0), ncol=1)
        plt.xticks(rotation=45)
        meu_grafico = plt.gcf()
        plt.close()
        self._atualizar_dataframe(totalFaturado, extras = [{'nome':'grafico', 'objeto':meu_grafico}] )        
    
    def faturamento_praca_mes(self):
        desempenho = self._filtrar_desempenho()
        totalFaturado = desempenho[['ano_mes_cobranca','faturamento_liquido', 'descricao_praca']].groupby(['ano_mes_cobranca', 'descricao_praca']).sum()
        lista_graficos = []
        for praca in desempenho['descricao_praca'].unique():
            dados_praca = desempenho[desempenho['descricao_praca'] == praca]
            totalFaturado = dados_praca[['ano_mes_cobranca', 'faturamento_liquido']].groupby('ano_mes_cobranca').sum()
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(totalFaturado.index, totalFaturado['faturamento_liquido'], marker='o', linestyle='-', label=praca)
            ax.set_title(f'Faturamento Líquido ao Longo do Tempo - {praca}')
            ax.set_xlabel('Ano e Mês de Cobrança')
            ax.set_ylabel('Faturamento Líquido')
            ax.grid(True)
            plt.xticks(rotation=45)
            plt.tight_layout()
            x = np.arange(len(totalFaturado.index))
            y = totalFaturado['faturamento_liquido']
            coefficients = np.polyfit(x, y, 1)
            trendline = np.poly1d(coefficients)
            ax.plot(totalFaturado.index, trendline(x), linestyle='--', color='red', label='Linha de Tendência')
            ax.legend()
            meu_grafico = plt.gcf()
            plt.close() 
            lista_graficos.append(meu_grafico)
        self._atualizar_dataframe(totalFaturado, extras = [{'nome':'grafico', 'objeto':lista_graficos}])
        
    def faturamento_promotor_praca_mes(self):
        desempenho = self._filtrar_desempenho()
        totalFaturado = desempenho[['ano_mes_cobranca','faturamento_liquido','descricao_vendedor', 'descricao_praca']].groupby(['ano_mes_cobranca', 'descricao_vendedor', 'descricao_praca']).sum()
        self._atualizar_dataframe(totalFaturado)

In [ ]:
class artigosApp(app):
    def __init__(self, main):
        super().__init__(main)        
        self.listaFuncoes = [('Deixado e Vendido Geral', 'deixado_vendido_simples'),
                            ('Deixado e Vendido Mensal', 'deixado_vendido_mes'),
                            ('Deixado e Vendido (Promotor/Mês)' ,'deixado_vendido_promotor_mes'),
                            ('Deixado e Vendido (Praça/Mês)' ,'deixado_vendido_praca_mes'),
                            ('Deixado e Vendido (Promotor/Periodo)' ,'deixado_vendido_periodo_promotor'),
                            ('Deixado, Vendido, Na rua (Geral)', 'deixado_vendido_simples_digitado'),
                            ('Deixado, Vendido, Na rua (Promotores)','deixado_vendido_simples_digitado_promotor'),
                            ('Deixado, Vendido e Faturado', 'deixado_vendido_faturado'),
                            ('Deixado, Vendido por mês', 'deixado_vendido_mensal'),
                            ('Deixado, Vendido, Rua e Score', 'deixado_vendido_rua_score'),
                            ('Variações por periodo','variacao_periodo'),
                            ]
        
    def deixado_vendido_simples(self):
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df = df.reset_index()
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        ordem = ['ref', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida']
        df = df.reindex(columns=ordem)
        df.sort_values(by='artigo', inplace= True)
        self._atualizar_dataframe(df)
        
    def deixado_vendido_mes(self):
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'ano_mes_cobranca']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        df.sort_values(by='artigo', inplace= True)
        agrupado = df.groupby('ano_mes_cobranca').agg({'quantidade_entregue': 'sum', 'quantidade_vendida': 'sum', '% Vendida': 'mean'}).reset_index()
        fig, ax1 = plt.subplots(figsize=(10, 6))
        x = np.arange(len(agrupado))
        y1 = agrupado['quantidade_entregue']
        ax1.plot(agrupado['ano_mes_cobranca'], y1, label='Quantidade Entregue', marker='o', linestyle='-', color='blue')
        coefficients1 = np.polyfit(x, y1, 1)
        trendline1 = np.poly1d(coefficients1)
        ax1.plot(agrupado['ano_mes_cobranca'], trendline1(x), linestyle='--', color='blue', label='Linha de Tendência (Quantidade Entregue)')
        y2 = agrupado['quantidade_vendida']
        ax1.plot(agrupado['ano_mes_cobranca'], y2, label='Quantidade Vendida', marker='s', linestyle='-', color = 'green')
        coefficients2 = np.polyfit(x, y2, 1)
        trendline2 = np.poly1d(coefficients2)
        ax1.plot(agrupado['ano_mes_cobranca'], trendline2(x), linestyle='--', color='green', label='Linha de Tendência (Quantidade Vendida)')
        ax1.set_xlabel('Ano e Mês de Cobrança')
        ax1.set_ylabel('Valores (Entregue e Vendido)')
        ax1.grid(True)
        ax1.legend(loc='upper left')
        plt.xticks(rotation=45)
        ax2 = ax1.twinx()
        y3 = agrupado['% Vendida']
        ax2.plot(agrupado['ano_mes_cobranca'], y3, label='% Vendida', marker='^', linestyle='-', color='black')
        coefficients3 = np.polyfit(x, y3, 1)
        trendline3 = np.poly1d(coefficients3)
        ax2.plot(agrupado['ano_mes_cobranca'], trendline3(x), linestyle='--', color='black', label='Linha de Tendência (% Vendida)')
        ax2.set_ylabel('% Vendida', color='red')
        ax2.tick_params(axis='y', labelcolor='red')
        ax2.legend(loc='upper right')
        plt.tight_layout()
        meu_grafico = plt.gcf()
        plt.close()
        self._atualizar_dataframe(df, extras = [{'nome':'grafico', 'objeto':meu_grafico}])
        
    def deixado_vendido_promotor_mes(self):
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'descricao_vendedor' ,'ano_mes_cobranca']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        df.sort_values(by='artigo', inplace= True)
        dfSI = df.reset_index()
        lista_graficos = []
        for vendedor in dfSI['descricao_vendedor'].unique():
            try:
                df_vendedor = dfSI[dfSI['descricao_vendedor'] == vendedor]
                agrupado = df_vendedor.groupby('ano_mes_cobranca').agg({
                    'quantidade_entregue': 'sum',
                    'quantidade_vendida': 'sum',
                    '% Vendida': 'mean'
                }).reset_index()
                fig, ax1 = plt.subplots(figsize=(10, 6))
                x = np.arange(len(agrupado))
                y1 = agrupado['quantidade_entregue']
                ax1.set_title(f'Gráfico para o Vendedor: {vendedor}')
                ax1.plot(agrupado['ano_mes_cobranca'], y1, label='Quantidade Entregue', marker='o', linestyle='-', color='blue')
                coefficients1 = np.polyfit(x, y1, 1)
                trendline1 = np.poly1d(coefficients1)
                ax1.plot(agrupado['ano_mes_cobranca'], trendline1(x), linestyle='--', color='blue', label='Linha de Tendência (Quantidade Entregue)')
                y2 = agrupado['quantidade_vendida']
                ax1.plot(agrupado['ano_mes_cobranca'], y2, label='Quantidade Vendida', marker='s', linestyle='-', color='green')
                coefficients2 = np.polyfit(x, y2, 1)
                trendline2 = np.poly1d(coefficients2)
                ax1.plot(agrupado['ano_mes_cobranca'], trendline2(x), linestyle='--', color='green', label='Linha de Tendência (Quantidade Vendida)')
                ax1.set_xlabel('Ano e Mês de Cobrança')
                ax1.set_ylabel('Valores (Entregue e Vendido)')
                ax1.grid(True)
                ax1.legend(loc='upper left')
                plt.xticks(rotation=45)
                ax2 = ax1.twinx()
                y3 = agrupado['% Vendida']
                ax2.plot(agrupado['ano_mes_cobranca'], y3, label='% Vendida', marker='^', linestyle='-', color='black')
                coefficients3 = np.polyfit(x, y3, 1)
                trendline3 = np.poly1d(coefficients3)
                ax2.plot(agrupado['ano_mes_cobranca'], trendline3(x), linestyle='--', color='black', label='Linha de Tendência (% Vendida)')
                ax2.set_ylabel('% Vendida', color='red')
                ax2.tick_params(axis='y', labelcolor='red')
                ax2.legend(loc='upper right')
                plt.tight_layout()
                meu_grafico = plt.gcf()
                lista_graficos.append(meu_grafico)
                plt.close()
            except:
                pass
        
        self._atualizar_dataframe(df, extras = [{'nome':'grafico', 'objeto':lista_graficos}])
        
    def deixado_vendido_praca_mes(self):
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'descricao_praca' ,'ano_mes_cobranca']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        df.sort_values(by='artigo', inplace= True)
        dfSI = df.reset_index()
        lista_graficos = []
        for praca in dfSI['descricao_praca'].unique():
            try:
                df_praca = dfSI[dfSI['descricao_praca'] == praca]
                agrupado = df_praca.groupby('ano_mes_cobranca').agg({
                    'quantidade_entregue': 'sum',
                    'quantidade_vendida': 'sum',
                    '% Vendida': 'mean'
                }).reset_index()
                fig, ax1 = plt.subplots(figsize=(10, 6))
                x = np.arange(len(agrupado))
                y1 = agrupado['quantidade_entregue']
                ax1.set_title(f'Gráfico para o praca: {praca}')
                ax1.plot(agrupado['ano_mes_cobranca'], y1, label='Quantidade Entregue', marker='o', linestyle='-', color='blue')
                coefficients1 = np.polyfit(x, y1, 1)
                trendline1 = np.poly1d(coefficients1)
                ax1.plot(agrupado['ano_mes_cobranca'], trendline1(x), linestyle='--', color='blue', label='Linha de Tendência (Quantidade Entregue)')
                y2 = agrupado['quantidade_vendida']
                ax1.plot(agrupado['ano_mes_cobranca'], y2, label='Quantidade Vendida', marker='s', linestyle='-', color='green')
                coefficients2 = np.polyfit(x, y2, 1)
                trendline2 = np.poly1d(coefficients2)
                ax1.plot(agrupado['ano_mes_cobranca'], trendline2(x), linestyle='--', color='green', label='Linha de Tendência (Quantidade Vendida)')
                ax1.set_xlabel('Ano e Mês de Cobrança')
                ax1.set_ylabel('Valores (Entregue e Vendido)')
                ax1.grid(True)
                ax1.legend(loc='upper left')
                plt.xticks(rotation=45)
                ax2 = ax1.twinx()
                y3 = agrupado['% Vendida']
                ax2.plot(agrupado['ano_mes_cobranca'], y3, label='% Vendida', marker='^', linestyle='-', color='black')
                coefficients3 = np.polyfit(x, y3, 1)
                trendline3 = np.poly1d(coefficients3)
                ax2.plot(agrupado['ano_mes_cobranca'], trendline3(x), linestyle='--', color='black', label='Linha de Tendência (% Vendida)')
                ax2.set_ylabel('% Vendida', color='red')
                ax2.tick_params(axis='y', labelcolor='red')
                ax2.legend(loc='upper right')
                plt.tight_layout()
                meu_grafico = plt.gcf()
                lista_graficos.append(meu_grafico)
                plt.close()
            except:
                pass
        
        self._atualizar_dataframe(df, extras = [{'nome':'grafico', 'objeto':lista_graficos}])
    
    def deixado_vendido_simples_digitado(self):
        artigosNoPedido = self._filtrar_artigosNoPedido(status="FECHADO")
        totalDigitado = self._filtrar_artigosNoPedido(status= "DIGITADO")
        totalDigitado = totalDigitado.groupby(['ref', 'artigo']).agg({'quantidade_entregue':'sum'})
        totalDigitado.rename(columns={'quantidade_entregue': 'Na Rua'}, inplace=True)
        df = artigosNoPedido.groupby(['ref', 'artigo']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df = pd.merge(df, totalDigitado[['Na Rua']], on=['ref', 'artigo'])
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        self._atualizar_dataframe(df)
        
    def deixado_vendido_simples_digitado_promotor(self):
        artigosNoPedido = self._filtrar_artigosNoPedido(status="FECHADO")
        totalDigitado = self._filtrar_artigosNoPedido(status= "DIGITADO")
        totalDigitado = totalDigitado.groupby(['ref', 'artigo', 'descricao_vendedor']).agg({'quantidade_entregue':'sum'})
        totalDigitado.rename(columns={'quantidade_entregue': 'Na Rua'}, inplace=True)
        df = artigosNoPedido.groupby(['ref', 'artigo', 'descricao_vendedor']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df = pd.merge(df, totalDigitado[['Na Rua']], on=['ref', 'artigo', 'descricao_vendedor'])
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        self._atualizar_dataframe(df)
        
    def deixado_vendido_mensal(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'ano_mes_cobranca']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        ordem = ['ref', 'ano_mes_cobranca', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida']
        self._atualizar_dataframe(df)
        
    def deixado_vendido_mensal_promotor(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'descricao_vendedor', 'ano_mes_cobranca']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        ordem = ['ref', 'ano_mes_cobranca', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida']
        self._atualizar_dataframe(df)
        
    def deixado_vendido_periodo_promotor(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'descricao_vendedor']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        ordem = ['ref', 'ano_mes_cobranca', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida']
        self._atualizar_dataframe(df)
    
    def deixado_vendido_rua_score(self):
        self._atualizar_filtros()
        artigosNoPedido = self._filtrar_artigosNoPedido()
        totalDigitado = self._filtrar_artigosNoPedido(status="DIGITADO")[['ref', 'quantidade_entregue']]
        totalDigitado = totalDigitado.groupby('ref').sum().reset_index()
        totalDigitado.rename(columns={'quantidade_entregue': 'Na Rua'}, inplace=True)
        df = artigosNoPedido.groupby('ref').sum()
        df = pd.merge(df, totalDigitado[['ref', 'Na Rua']], on='ref')
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        df['Score'] = df['quantidade_vendida'] * df['% Vendida']
        ordem = ['ref', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida', 'Na Rua','Score']
        df = df.reindex(columns=ordem)
        df = df.sort_values('Score', ascending= False)
        self._atualizar_dataframe(df)
    
    def deixado_vendido_faturado(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum', 'valor_vendido':'sum'})
        df["%_total_vendido"] = (df['valor_vendido']/df['valor_vendido'].sum()) * 100
        df = df.sort_values('valor_vendido', ascending= False)
        df['%_total_vendido_acumulado'] = df['%_total_vendido'].cumsum() - df['%_total_vendido'] + df['%_total_vendido'].iloc[0]
        
        condicoes = [
            (df['%_total_vendido_acumulado'] <= 50 ),
            (df['%_total_vendido_acumulado'] <= 80 ),
            (df['%_total_vendido_acumulado'] > 80)
        ]
        notas = ['A', 'B', 'C']
        df['rating'] = np.select(condicoes, notas)
        
        self._atualizar_dataframe(df)
        
    def variacao_periodo(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'ano_mes_cobranca']).sum().reset_index()
        df['giro'] = df['ano_mes_cobranca']
        meses = df['ano_mes_cobranca'].unique()
        grupos = [meses[i:i+2] for i in range(0, len(meses), 2)]
        giro = 0
        for grupo in grupos:
            giro += 1
            for data in grupo:
                df['giro'] = df['giro'].replace(data, giro)
                
        df = df.groupby(['ref', 'giro']).sum().reset_index()
        df['flutuacao_entregue'] = df.groupby('ref')['quantidade_entregue'].pct_change() * 100
        df['mudanca_total_entregue'] = df.groupby('ref')['flutuacao_entregue'].cumsum()
        df['flutuacao_vendida'] = df.groupby('ref')['quantidade_vendida'].pct_change() * 100
        df['mudanca_total_vendida'] = df.groupby('ref')['flutuacao_vendida'].cumsum()
        df['porcentagem_vendida'] = df['quantidade_vendida'] / df['quantidade_entregue'] * 100
        df['flutuacao_porcentagem_vendida'] = df.groupby('ref')['porcentagem_vendida'].pct_change() * 100
        df['mudanca_total_porcentagem_vendida'] = df.groupby('ref')['flutuacao_porcentagem_vendida'].cumsum()
        ordem = ['ref', 'quantidade_entregue','flutuacao_entregue', 'mudanca_total_entregue', 'quantidade_vendida', 'flutuacao_vendida',  'mudanca_total_vendida', 'porcentagem_vendida', 'flutuacao_porcentagem_vendida','mudanca_total_porcentagem_vendida']
        df = df.reindex(columns=ordem)
        nomes = {'quantidade_entregue': 'Entregue', 'quantidade_vendida': 'Vendida', 'flutuacao_entregue' : 'Mov. Entregue', 'mudanca_total_entregue': 'Mov.To Entregue', 'flutuacao_vendida': 'Mov. Vendida', 'mudanca_total_vendida': 'Mov.To Vendida', 'porcentagem_vendida': '% Vendida', 'flutuacao_porcentagem_vendida': 'Mov. % Vendida','mudanca_total_porcentagem_vendida': 'Mov.To % Vendida' }
        df.rename(columns= nomes, inplace = True)
        self._atualizar_dataframe(df)

In [ ]:
class MainApp:
    def __init__(self):
        self.app_artigos = artigosApp(self)
        self.app_vendedoras = vendedorasApp(self)
        self.app_desempenho = desempenhoApp(self)
        self.app_cidades = cidadesApp(self)
        self.apps = [
            ('Artigos', self.app_artigos),
            ('Desempenho', self.app_desempenho), 
            ('Cidades', self.app_cidades),
            ('Vendedoras', self.app_vendedoras)]
        self.load_data()
        self.create_layout()         
        self.tabs_data = []
        
    def load_data(self):
        self.vendedores = pd.read_csv('data/vendedores.csv', sep=';')
        self.pracas = pd.read_csv('data/pracas.csv', sep=';')
        self.desempenhoPraca = pd.read_csv('data/desempenhoPraca.csv', sep=';')
        self.pedidos = pd.read_csv('data/pedidos.csv', sep=';')
        self.sacoleiras = pd.read_csv('data/sacoleiras.csv', sep=';')
        self.artigosPedido = pd.read_csv('data/artigosPedido.csv', sep=';')
        self.gruposArtigos = pd.read_csv('data/grupos.csv', sep=';')
        self.artigos = pd.read_csv('data/artigos.csv', sep=';')
        self.infoCidades = pd.read_csv('data/cidades.csv', sep=';')
        
        self.artigosPedido['ano_mes_cobranca'] = pd.to_datetime(self.artigosPedido['ano_mes_cobranca'], format='%m/%Y').dt.date
        self.desempenhoPraca['ano_mes_cobranca'] = pd.to_datetime(self.desempenhoPraca['ano_mes_cobranca'], format='%Y-%m').dt.date
        self.pedidos['ano_mes_cobranca'] = pd.to_datetime(self.pedidos['ano_mes_cobranca'], format='%m/%Y').dt.date

    def create_layout(self):
        # Promotor
        self.check_soPromotorEPracaAtivo = widgets.Checkbox(description="Só Ativas?", value=True)
        listaPromotores = self.get_lista_promotores()
        self.select_promotores = widgets.SelectMultiple(description='Promotor(es)', options=listaPromotores, value=[promotor[1] for promotor in listaPromotores])
        self.check_todosPromotores = widgets.Checkbox(description="Todos", value=False)
        self.box_promotores = widgets.HBox(children=[self.select_promotores, self.check_todosPromotores])
        
        # Praça
        listaPracas = self.get_lista_pracas()
        self.text_id_pracas = widgets.Text(description='Praça(s) [id]')
        self.select_pracas = widgets.SelectMultiple(description='Praça(s)', options=listaPracas, value=[praca[1] for praca in listaPracas])
        self.check_todasPracas = widgets.Checkbox(description="Todas", value=False)
        self.box_pracas = widgets.HBox(children=[self.text_id_pracas, self.select_pracas, self.check_todasPracas])
        self.box_promotorPraca = widgets.HBox(children=[self.box_promotores, self.box_pracas, self.check_soPromotorEPracaAtivo])
        
        # Periodo
        self.dateP_dataInicial = widgets.DatePicker(description='Data Inicial Cobrança', value=dt.date.today() - dt.timedelta(days=2 * 30))
        self.dateP_dataFinal = widgets.DatePicker(description='Data Final Cobrança', value=dt.date.today() + dt.timedelta(days=2 * 30))
        self.box_periodo = widgets.HBox(children=[self.dateP_dataInicial, self.dateP_dataFinal])
        
        # Grupos e Artigos
        listaGrupos = self.get_lista_grupos()
        self.select_gruposArtigos = widgets.SelectMultiple(description='Grupos', options=listaGrupos, value=[grupo[1] for grupo in listaGrupos])
        self.check_todosGrupos = widgets.Checkbox(description='Todos', value=False)
        listaArtigos = self.get_lista_artigos()
        self.text_ref_artigos = widgets.Text(description='Artigos(s) [ref]')
        self.select_artigos = widgets.SelectMultiple(description='Artigos', options=listaArtigos, value=[artigo[1] for artigo in listaArtigos])
        self.check_todosArtigos = widgets.Checkbox(description='Todos', value=False)
        self.box_gruposArtigos = widgets.HBox(children=[self.select_gruposArtigos, self.check_todosGrupos,self.text_ref_artigos, self.select_artigos, self.check_todosArtigos])

        # Relatórios
        self.select_app = widgets.Dropdown(description='App:', options=self.apps)
        self.select_funcao = widgets.Dropdown(description='Função:', options=self.get_lista_funcoes())
        self.togle_modoExibicao = widgets.ToggleButtons(
            options=['Mesma Tab', 'Nova Tab'],
            description='Relatório na:',
            value='Mesma Tab',
            tooltips=['O relatório substituirá a tab atual', 'O relatório entrará em uma nova tab'],
        )
        self.button_rodar = widgets.Button(description= 'Rodar')
        self.button_rodar.on_click(self._rodar)
        self.button_fechar = widgets.Button(description= 'Fechar Tab')
        self.button_fechar.on_click(self.fecharTabAtiva)
        self.box_apps = widgets.HBox(children=[self.select_app, self.select_funcao, self.togle_modoExibicao, self.button_rodar, self.button_fechar])

        self.filtros = [self.box_promotorPraca, self.box_periodo, self.box_gruposArtigos]
        self.box_filtros = widgets.VBox(children=self.filtros)
        self.tab_apps = widgets.Tab()
        self.box_fixos = [self.box_filtros, self.box_apps, self.tab_apps]
        self.box_main = widgets.VBox(children=self.box_fixos)
        # Add observers to all widgets
        self.add_observer_to_widgets(self.box_filtros)
        # Display the nested boxes
        display(self.box_main)
    
    def get_lista_funcoes(self):
        return self.select_app.value.listaFuncoes # type: ignore

    def get_lista_promotores(self):
        listaPromotores = []
        promotores = self.vendedores.sort_values(by='descricao_vendedor', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            promotores = promotores.query('status_vendedor == "ATIVO"')
        for _, linha in promotores.iterrows():
            tupla = (linha['descricao_vendedor'], linha['id_vendedor'])
            listaPromotores.append(tupla)
        return listaPromotores
    
    #TODO Juntar função get_lista_pracas e get_lista_id_pracas
    def get_lista_pracas(self):
        listaPracas = []
        promotores = self.select_promotores.value
        pracas = self.pracas.query('id_vendedor.isin(@promotores)')
        pracas = pracas.sort_values(by='descricao_praca', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            pracas = pracas.query('status_praca == "ATIVO"')
        for _, linha in pracas.iterrows():
            tupla = (linha['descricao_praca'], linha['id_praca'])
            listaPracas.append(tupla)
        return listaPracas
    
    def get_lista_id_pracas(self):
        listaPracas = []
        promotores = self.select_promotores.value
        pracas = self.pracas.query('id_vendedor.isin(@promotores)')
        pracas = pracas.sort_values(by='descricao_praca', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            pracas = pracas.query('status_praca == "ATIVO"')
        for _, linha in pracas.iterrows():
            tupla = (linha['id_praca'])
            listaPracas.append(tupla)
        return listaPracas

    def get_lista_grupos(self):
        listaGrupos = []
        grupos = self.gruposArtigos
        grupos = grupos.sort_values(by='descricao_grupo', ascending=True)

        for _, linha in grupos.iterrows():
            tupla = (linha['descricao_grupo'], linha['id_grupo'])
            listaGrupos.append(tupla)

        return listaGrupos
    
    #TODO Juntar função get_lista_artigos e get_lista_ref_artigos
    def get_lista_artigos(self):
        listaArtigos = []
        grupos = self.select_gruposArtigos.value
        artigos = self.artigos.query('id_grupo.isin(@grupos)')
        artigos = artigos.sort_values(by='artigo', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            artigos = artigos.query('status_artigo == "ATIVO"')
        for _, linha in artigos.iterrows():
            tupla = (linha['artigo'], linha['ref'])
            listaArtigos.append(tupla)
        return listaArtigos
    
    def get_lista_ref_artigos(self):
        listaArtigos = []
        grupos = self.select_gruposArtigos.value
        artigos = self.artigos.query('id_grupo.isin(@grupos)')
        artigos = artigos.sort_values(by='artigo', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            artigos = artigos.query('status_artigo == "ATIVO"')
        for _, linha in artigos.iterrows():
            tupla = (linha['ref'])
            listaArtigos.append(tupla)
        return listaArtigos

    def add_observer_to_widgets(self, widget):
        # Specific functions
        self.select_promotores.observe(self._promotor_selecionado, names='value') # type: ignore
        self.select_gruposArtigos.observe(self._grupos_selecionados, names='value') # type: ignore
        self.check_todosPromotores.observe(self._selecionar_todos_promotores, names='value') # type: ignore
        self.check_todasPracas.observe(self._selecionar_todas_pracas, names='value') # type: ignore
        self.check_todosGrupos.observe(self._selecionar_todos_grupos, names='value') # type: ignore
        self.check_todosArtigos.observe(self._selecionar_todos_artigos, names='value') # type: ignore
        self.select_app.observe(self._app_selecionado, names='value') # type: ignore
        self.text_id_pracas.observe(self.id_praca_escrito, names='value') # type: ignore
        self.text_ref_artigos.observe(self.ref_artigo_escrito, names='value') # type: ignore
        
        # General function
        if isinstance(widget, (widgets.Box, widgets.VBox, widgets.HBox, widgets.GridBox)):
            for child_widget in widget.children: # type: ignore
                self.add_observer_to_widgets(child_widget)
        elif isinstance(widget, widgets.Widget):
            widget.observe(self.normal_object_updated, names='value') # type: ignore

    def normal_object_updated(self, change):
        if change['new'] == ():
            pass
        else:
            self._update_termos()

    def id_praca_escrito(self, change):
        valor = change['new']
        if valor == '':
            return
        padrao = re.compile(r'^[0-9,]*$')
        lista_pracas = self.get_lista_id_pracas()
        if padrao.match(valor):
            if valor[-1] != ',':
                pracasSelecionadasLista = [int(numero) for numero in valor.split(',')]
                pracasSelecionadas = []
                for praca in pracasSelecionadasLista:
                    if praca in lista_pracas:
                        pracasSelecionadas.append(praca)
                self.select_pracas.value = pracasSelecionadas
        else:
            self.text_id_pracas.value = change['old']
            
    def ref_artigo_escrito(self, change):
        valor = change['new']
        if valor == '':
            return
        padrao = re.compile(r'^[0-9,]*$')
        lista_artigos = self.get_lista_ref_artigos()
        if padrao.match(valor):
            if valor[-1] != ',':
                artigosSelecionadosLista = [int(numero) for numero in valor.split(',')]
                artigosSelecionados = []
                for praca in artigosSelecionadosLista:
                    if praca in lista_artigos:
                        artigosSelecionados.append(praca)
                self.select_artigos.value = artigosSelecionados
        else:
            self.text_ref_artigos.value = change['old']
        
        
    def _update_termos(self):
        # Convertendo as strings das datas para o formato "mm/aaaa"
        data_inicial_str = self.dateP_dataInicial.value.strftime("%m/%Y") # type: ignore
        data_final_str = self.dateP_dataFinal.value.strftime("%m/%Y") # type: ignore
        # Convertendo as strings para o tipo datetime
        data_inicial = datetime.strptime(data_inicial_str, "%m/%Y").date()
        data_final = datetime.strptime(data_final_str, "%m/%Y").date()
        self.filtros = {
            'promotores': self.select_promotores.value,
            'pracas': self.select_pracas.value,
            'data inicial': data_inicial,
            'data final': data_final,
            'artigos': self.select_artigos.value
        }
        
    def set_funcao(self):
        funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
        funcao_selecionada()

    def _updateOptionsPracas(self):
        self.select_pracas.options = self.get_lista_pracas()
        self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]

    def _updateOptionsArtigos(self):
        self.select_artigos.options = self.get_lista_artigos()
        self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]

    def _updateOptionsFuncoes(self):
        self.select_funcao.options = self.get_lista_funcoes()

    def _promotor_selecionado(self, change):
        self._updateOptionsPracas()

    def _grupos_selecionados(self, change):
        self._updateOptionsArtigos()

    def _app_selecionado(self, change):
        self._updateOptionsFuncoes()
        self._update_termos()

    def _rodar(self, _):
        self._update_termos()
        if self.togle_modoExibicao.value == 'Mesma Tab':
            self.rodarAppNaTabAtiva()
        elif self.togle_modoExibicao.value == 'Nova Tab':
            self.rodarAppEmNovaTab()
    
    def rodarAppNaTabAtiva(self):
        indice_tab_ativa = self.tab_apps.selected_index
        
        # Verifica se há alguma tab aberta
        if indice_tab_ativa is not None and indice_tab_ativa >= 0:
            tab_ativa = self.tab_apps.children[indice_tab_ativa] # type: ignore
            selected_app = self.select_app.value
            app_widgets = selected_app._build_layout() # type: ignore
            tab_ativa.children = [] + [app_widgets]
            funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
            funcao_selecionada()
            self.tab_apps.set_title(len(self.tab_apps.children) - 1, f'{self.select_funcao.label}') # type: ignore
        else:
            self.rodarAppEmNovaTab()
        
    def rodarAppEmNovaTab(self):
        selected_app = self.select_app.value
        app_widgets = selected_app._build_layout() # type: ignore
        nova_aba = widgets.VBox([app_widgets])
        self.tab_apps.children = list(self.tab_apps.children) + [nova_aba] # type: ignore
        self.tab_apps.set_title(len(self.tab_apps.children) - 1, f'{self.select_funcao.label}')
        funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
        funcao_selecionada()

    def fecharTabAtiva(self, _):
        indice_tab_ativa = self.tab_apps.selected_index
        if indice_tab_ativa is not None and indice_tab_ativa >= 0:
            self.tab_apps.children = [
            child for i, child in enumerate(self.tab_apps.children) if i != indice_tab_ativa # type: ignore
        ]
        
    def _selecionar_todos_promotores(self, change):
        if change['new']:
            self.select_promotores.value = [promotor[1] for promotor in self.get_lista_promotores()]
        else:
            self.select_promotores.value = []

    def _selecionar_todas_pracas(self, change):
        if change['new']:
            self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]
        else:
            self.select_pracas.value = []

    def _selecionar_todos_grupos(self, change):
        if change['new']:
            self.select_gruposArtigos.value = [grupo[1] for grupo in self.get_lista_grupos()]
        else:
            self.select_gruposArtigos.value = []

    def _selecionar_todos_artigos(self, change):
        if change['new']:
            self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]
        else:
            self.select_artigos.value = []


In [ ]:
app = MainApp()
